# Semantic Link Data Preparation - Ship Date Prediction

**Goal:** Prepare data from the "DLV Aging Columns & Measures" semantic model to predict when orders will ship from the distribution center.

## Use Case
Predict **when open deliveries will ship from the DC** (GI Date), enabling:
- Proactive lead time forecasting for business teams
- Better inventory and logistics planning
- Identification of deliveries likely to have extended ship times
- Improved customer communication about expected ship dates

## This Notebook Will:
1. Connect to the semantic model
2. Load historical **closed deliveries** (last 2-4 weeks) for training
3. Validate the target variable: `DAYS_TO_SHIP` (days from creation to DC ship date)
4. Explore features: Plant, Brand, Channel, Carrier, Strategic Account, etc.
5. Prepare data for AutoML training

### 📦 1. Install Semantic Link

In [ ]:
%pip install -U semantic-link --q

### 🔧 2. Configuration

**IMPORTANT:** Update the semantic model name to match your environment.

In [ ]:
# ==============================================================================
# CONFIGURATION: Semantic Model Connection
# ==============================================================================
# This cell connects to your Power BI semantic model using Semantic Link.
# 
# WHY: Semantic Link allows Python to query Power BI semantic models directly,
#      avoiding data duplication and ensuring consistency with your reports.
#
# WHAT YOU NEED TO UPDATE:
# - DATASET: Change to match your exact semantic model name in Fabric
# ==============================================================================

import sempy.fabric as fabric 
import pandas as pd
from sempy.fabric import FabricDataFrame

# Semantic model name (UPDATE THIS to match your environment)
DATASET = "DLV Aging Columns & Measures"

# Get workspace ID (automatically detects current Fabric workspace)
ws = fabric.get_workspace_id()

print(f"✅ Workspace ID: {ws}")
print(f"✅ Semantic Model: {DATASET}")

### 📊 3. List Tables in Semantic Model

Let's see what tables are available in the semantic model.

In [ ]:
# List all tables in the semantic model
# WHY: Verify the semantic model is accessible and see available tables
tables_fdf = fabric.list_tables(DATASET, workspace=ws) 
print(f"Tables found: {len(tables_fdf)}")
tables_fdf


### 📋 4. List Columns in the Aging Table

Explore all columns available in the Aging table.

In [ ]:
# List all columns across all tables
# WHY: See what data is available in the semantic model
# NOTE: This shows table columns only - DAX measures are NOT included
columns_df = fabric.list_columns(DATASET, workspace=ws)
print(f"\nTotal columns across all tables: {len(columns_df)}")
print(f"\nColumn details:")
columns_df


### 🔗 5. Visualize Relationships

Check if there are any relationships between tables in the semantic model.

In [ ]:
# Check for relationships between tables
# WHY: Understanding table relationships helps with data modeling
# NOTE: Many semantic models have a single denormalized table (no relationships)
try:
    from sempy.relationships import plot_relationship_metadata
    relationships = fabric.list_relationships(workspace=ws, dataset=DATASET)
    
    if len(relationships) > 0:
        print(f"Found {len(relationships)} relationships")
        plot_relationship_metadata(relationships)
    else:
        print("No relationships found in this semantic model.")
        print("The Aging table appears to be a single fact table with all data.")
except Exception as e:
    print(f"Note: {e}")
    print("This is normal if the Aging table is a flat/denormalized table.")


### 📥 6. Load Closed Deliveries for Training

In [ ]:
# ==============================================================================
# LOAD TRAINING DATA: Closed Deliveries Only
# ==============================================================================
# WHY: Machine learning requires labeled data (known outcomes).
#      Closed deliveries have GI Date (ship date), so we can calculate
#      how long it took from creation to ship (DAYS_TO_SHIP).
#
# FILTER: NOT(ISBLANK(Aging[GI Date])) AND NOT(ISBLANK(Aging[Delivery Created On]))
#         This ensures we only get deliveries that have both creation and ship dates.
#
# NOTE: Open deliveries (no GI Date yet) will be scored in Notebook 03.
# ==============================================================================

dax_query = """
EVALUATE
FILTER(
    Aging,
    NOT(ISBLANK(Aging[GI Date])) &&
    NOT(ISBLANK(Aging[Delivery Created On]))
)
"""

df_closed = fabric.evaluate_dax(dataset=DATASET, dax_string=dax_query, workspace=ws)

# Clean column names (DAX adds table prefixes like 'Aging[column]')
df_closed.columns = [col.split('[')[-1].replace(']', '') if '[' in col else col for col in df_closed.columns]

print(f"✅ Loaded {len(df_closed):,} closed deliveries")
print(f"✅ Columns: {df_closed.shape[1]}")
df_closed.head()

### 🔍 7. Quick Data Check

Verify that required date columns are present.

### 🎯 9. Validate Target Variable: DAYS_TO_SHIP

**Target Variable:** `DAYS_TO_SHIP` - Days from delivery creation to actual ship date from DC

Calculated as: `GI Date - Delivery Created On`

- **Higher values** = Longer lead time (more days to ship)
- **Lower values** = Faster processing (fewer days to ship)

This is the key metric for:
- Forecasting when orders will ship from the distribution center
- Planning logistics and inventory allocation
- Setting realistic customer expectations for ship dates
- Identifying orders with unusually long processing times

In [ ]:
# ==============================================================================
# VALIDATE TARGET VARIABLE: DAYS_TO_SHIP
# ==============================================================================
# WHY: This is what we're trying to predict - how many days from creation
#      to ship from the distribution center.
#
# DAYS_TO_SHIP = Days between Delivery Created On and GI Date (actual ship)
#
# We need both columns to calculate this:
# - Delivery Created On: When the delivery was created in the system
# - GI Date: When the delivery actually shipped from the DC
# ==============================================================================

# Check if required columns exist
required_cols = ['GI Date', 'Delivery Created On']
missing = [col for col in required_cols if col not in df_closed.columns]

if missing:
    print(f"⚠️ MISSING REQUIRED COLUMNS: {missing}")
    print(f"Available columns: {list(df_closed.columns)}")
else:
    # Calculate DAYS_TO_SHIP
    df_closed['DAYS_TO_SHIP'] = (
        pd.to_datetime(df_closed['GI Date']) - 
        pd.to_datetime(df_closed['Delivery Created On'])
    ).dt.days
    
    days_data = df_closed['DAYS_TO_SHIP'].dropna()
    
    print("="*60)
    print("TARGET VARIABLE: DAYS_TO_SHIP")
    print("="*60)
    print(f"Total records: {len(days_data):,}")
    print(f"Mean: {days_data.mean():.2f} days")
    print(f"Median: {days_data.median():.2f} days")
    print(f"Min: {days_data.min():.0f} days")
    print(f"Max: {days_data.max():.0f} days")
    print(f"\nDistribution:")
    print(f"  0-2 days: {(days_data <= 2).sum():,} ({(days_data <= 2).sum()/len(days_data)*100:.1f}%)")
    print(f"  3-5 days: {((days_data > 2) & (days_data <= 5)).sum():,} ({((days_data > 2) & (days_data <= 5)).sum()/len(days_data)*100:.1f}%)")
    print(f"  6-9 days: {((days_data > 5) & (days_data <= 9)).sum():,} ({((days_data > 5) & (days_data <= 9)).sum()/len(days_data)*100:.1f}%)")
    print(f"  10+ days: {(days_data > 9).sum():,} ({(days_data > 9).sum()/len(days_data)*100:.1f}%)")
    print("="*60)

### ✅ 8. Summary

Data is ready for model training!

In [ ]:
print("="*60)
print("DATA PREPARATION COMPLETE")
print("="*60)

# Validate target
required_cols = ['GI Date', 'Delivery Created On']
missing = [col for col in required_cols if col not in df_closed.columns]

if missing:
    print(f"\n❌ ERROR: Missing columns for target calculation: {missing}")
else:
    print("\n✅ TARGET VARIABLE: DAYS_TO_SHIP")
    print("   (Calculated from GI Date - Delivery Created On)")

# List available features
recommended_features = [
    'Plant', 'Brand', 'Channel', 'Product Category', 'Product Type',
    'Standard Or Custom', 'Credit Status', 'Distribution Status',
    'STRATEGIC_ACCOUNT', 'EWM Carrier Code', 'Shipping Point',
    'Delivery Type', 'DELIVERY_QTY', 'DELIVERY_VALUE_USD'
]

available = [f for f in recommended_features if f in df_closed.columns]
print(f"\n✅ Available features: {len(available)}/{len(recommended_features)}")
print(f"✅ Total records: {len(df_closed):,}")

print("\n" + "="*60)
print("NEXT: Open 02_autoML_training_pipeline.ipynb")
print("="*60)

---

## Next Step

Proceed to **`02_autoML_training_pipeline.ipynb`** to train the ship date prediction model.

The model will predict:
- **DAYS_TO_SHIP** (regression): How many days from creation to DC ship date?
- **predicted_ship_date**: Calculated as Delivery Created On + predicted days to ship